# Assemble green function with AE event:  remove response
2022.02.15 Kurama Okubo

2022.05.08 update for fb03-045

2022.07.29 update for removal response using PAZ

2022.12.05 update for removal response with preamp, AE sensor and the source factor obtained from the ball drop.

2023.07.24 update for fb03-087

2023.10.22 update for fb03-087 multiple events

2024.2.6 update for master plot: replaced the ball drop gain model to model 4 including the aperture effect

2024.6.9 update save different stream with more sensors.

2024.10.31 update for the merged catalog

## Work flow

1. Read isocoord pickle associated with stations
2. Read meta data
3. Read synthetic waveform
4. Preprocess (unscale using gain to the observation **& remove response**)
4. Save pickle

In [ ]:
import os
import obspy
from obspy import read, Stream, Trace
from obspy.signal.invsim import paz_to_freq_resp

from scipy import signal
import scipy.io as sio
import matplotlib.pyplot as plt
%matplotlib inline
import glob
from glob import glob
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from tqdm import tqdm
import warnings

%load_ext autoreload
%autoreload 2

from remove_resp_digitalfilt import *

os.environ['TZ'] = 'GMT' # change time zone to avoid confusion in unix_tvec conversion

plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 12
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 5
plt.rcParams["xtick.major.width"] = 0.5
plt.rcParams["xtick.minor.size"] = 2
plt.rcParams["xtick.minor.width"] = 1
plt.rcParams["xtick.minor.visible"] = True


plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 5
plt.rcParams["ytick.major.width"] = 0.5
plt.rcParams["ytick.minor.size"] = 2
plt.rcParams["ytick.minor.width"] = 1
plt.rcParams["ytick.minor.visible"] = True

    
os.environ['TZ'] = 'GMT' # change time zone to avoid confusion in unix_tvec conversion

from obspy.core.utcdatetime import UTCDateTime  
UTCDateTime.DEFAULT_PRECISION = 8 # increase the time precision


In [ ]:
#Parameters
simulationtitle = 'MTinv_fb03-087' # to read sac, set simulation title used in Input.inf of OpenSWPC

balldrop_model=4 # 2 for the model only with SiTj, 4 for the SiTjbeta

# rootdir for removal resp analysis
rootdir = "./"
# Sensor gain
fi_balldropgain = rootdir+f'../../Calibration/SensorCoupling_BallDrop/data/DATA_surfaceeffect_Aij_Case2/master_gain_model{balldrop_model}.csv'
# fi_balldropgain = rootdir+f'../data/resp/master_gain_model3.csv' # debug
fi_AEresp = rootdir+'../../Calibration/AEsensor_Calibration/AEsensor_Calibration_ARX/data/AE_resp_dataandcoef_fronttop.mat'
# fi_AEresp = rootdir+'../data/resp/debug_AE_resp_dataandcoef_fronttop.mat' # debug
fi_preamp = rootdir+'../data/resp/preamp_mistras_resp.mat'
figdir = "../figure"

fontsize = 12
vmean = 6200
fs = 1e7; # sampling frequency

# Pre-amplification
preamp_dB = 40 # [dB]
preamp_V = 10 ** (preamp_dB/20)
preamp_V

# Test of removal response using metal block test
We first test to remove the response using metal block data.
The coefficients of PAZ are estimated using matlab code.

In [ ]:
D = sio.loadmat(fi_AEresp);
D.keys()

In [ ]:
np.squeeze(D["u_input_raw"])

In [ ]:
# compile trace
st_mb = Stream()
tr_u = Trace()
tr_u.stats.starttime = datetime.datetime(2022,7,29)
tr_u.stats.sampling_rate = fs
tr_u.stats.network = "N"
tr_u.stats.station = "IN"
tr_u.stats.channel = "Z"
tr_u.data = np.squeeze(D["u_input_raw"])

tr_y = tr_u.copy()
tr_y.stats.station = "OUT"
tr_y.stats.channel = "Z"
tr_y.data = np.squeeze(D["y_output_raw"])


In [ ]:
tr_y.stats

In [ ]:
# remove response using the function
poles_AE = np.squeeze(D["p"])
zeros_AE = np.squeeze(D["z"])
scale_fac_AE = np.squeeze(D["k"])
u_normfact_AE = np.squeeze(D["u_normfact"])
pre_filt = (1e4, 2e4, 1e6, 2e6)
water_level = 60

# debug
# scale_fac_AE /= u_normfact_AE

tr_removed = remove_resp_digitalfilt(tr_y, poles_AE, zeros_AE, scale_fac_AE, pre_filt=pre_filt, water_level=60, zero_mean=True,
    taper=True, taper_fraction=0.05, detrend=True, debug=False)

In [ ]:
freqmin = 6e4
freqmax = 3e5

tr_u_filtered = tr_u.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax, corners=3, zerophase=True)
tr_inv_filtered = tr_removed.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax, corners=3, zerophase=True)
tr_y_filtered =  tr_y.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax, corners=3, zerophase=True)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
tvec = tr_u_filtered.times()*1e6
ax.plot(tvec, tr_u_filtered.data, "k-", label="LDV")
ax.plot(tvec, tr_inv_filtered.data, "r-", label="inverted")
# ax.plot(tvec, tr_y_filtered.data/5e4, "b--", label="inverted")
ax.set_xlim([0, 100])



In [ ]:
freqmin = 1e4
freqmax = 2e5
freqmax_high = 4e5

fig, axs = plt.subplots(3, 2, figsize=(16, 10))
tvec = tr_u_filtered.times()*1e6

axs[0,0].plot(tvec, tr_u.data*1e3, "k-", label="Raw LDV unfiltered")
axs[0,0].set_ylim([-0.6, 0.6])
axs[0,0].set_ylabel("mm/s")

axs[0,1].plot(tvec, tr_y.data, "r-", label="Raw AE sensor unfiltered")
axs[0,1].set_ylim([-5.0, 5.0])
axs[0,1].set_ylabel("Volt [V]")

axs[1,0].plot(tvec, tr_u.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax, corners=3, zerophase=True).data*1e3, "k-", label=f"LDV {int(freqmin/1e3)}kHz-{int(freqmax/1e3)}kHz")
axs[1,0].set_ylim([-0.1, 0.1])
axs[1,0].set_ylabel("mm/s")

axs[1,1].plot(tvec, tr_removed.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax, corners=3, zerophase=True).data*1e3, "r-",
                                          label=f"{int(freqmin/1e3)}kHz-{int(freqmax/1e3)}kHz after AE response removed")
axs[1,1].set_ylim([-0.1, 0.1])
axs[1,1].set_ylabel("mm/s")

axs[2,0].plot(tvec, tr_u.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax, corners=3, zerophase=True).data*1e3, "k-", label=f"LDV {int(freqmin/1e3)}kHz-{int(freqmax/1e3)}kHz")
axs[2,0].plot(tvec, tr_removed.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax, corners=3, zerophase=True).data*1e3, "r-",label=f"{int(freqmin/1e3)}kHz-{int(freqmax/1e3)}kHz after AE response removed")
axs[2,0].set_ylim([-0.1, 0.1])
axs[2,0].set_ylabel("mm/s")

axs[2,1].plot(tvec, tr_u.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax_high, corners=3, zerophase=True).data*1e3, "k-", label=f"LDV  {int(freqmin/1e3)}kHz-{int(freqmax_high/1e3)}kHz")
axs[2,1].plot(tvec, tr_removed.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax_high, corners=3, zerophase=True).data*1e3, "r-",label=f" {int(freqmin/1e3)}kHz-{int(freqmax_high/1e3)}kHz after AE response removed")
axs[2,1].set_ylim([-0.2, 0.2])
axs[2,1].set_ylabel("mm/s")


for ax_h in axs:
    for ax in ax_h:
        ax.legend(loc=1)
        ax.set_xlim([0, 100])
        
plt.tight_layout()
# plt.savefig(figdir+"/LDVandAE_comparison.png", dpi=300)


## Plot poles and zeros

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

poles_AE.size, np.count_nonzero(zeros_AE)

ax.scatter([np.real(x) for x in zeros_AE], [np.imag(x) for x in zeros_AE], label="zeros",
           marker="x", c="r")
ax.scatter([np.real(x) for x in poles_AE], [np.imag(x) for x in poles_AE], label="poles",
           marker="o", fc="None", ec="k")
ax.set_xlim([-1.2, 1.2])
ax.set_ylim([-1.2, 1.2])
ax.set_aspect('equal')
ax.legend(loc=1)
ax.set_title(f"Npoles:{poles_AE.size} Nzeros:{np.count_nonzero(zeros_AE)}")

# Bode plot

In [ ]:
_, freqs, freq_domain_taper, data_after_freqtapered, freq_response_forward, freq_response = remove_resp_digitalfilt(tr_y, poles_AE, zeros_AE, scale_fac_AE, pre_filt=pre_filt, water_level=60, zero_mean=True,
    taper=True, taper_fraction=0.05, detrend=True, debug=True)

In [ ]:
len(freqs)

In [ ]:
# compute G from just dividing y from u
from obspy.signal.util import _npts2nfft
nfft = _npts2nfft(tr_y.stats.npts)
U = np.fft.rfft(tr_u.data, n=nfft)
Y = np.fft.rfft(tr_y.data, n=nfft)
g_freqs = np.fft.rfftfreq(nfft, d=1/fs)
g_est = Y/U

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 6))
xlimit = [10e3, 5e6]

axs[0].grid(True, which="both", ls="-", alpha=0.2)

axs[0].semilogx(g_freqs, 20 * np.log10(abs(g_est)), "k-", label="Estimated with Y/U")
axs[0].semilogx(freqs, 20 * np.log10(abs(freq_response_forward)), "r-", label="Modeled with Poles and Zeros")

axs[0].set_xlabel("Frequency [Hz]")
axs[0].set_ylabel("Amplitude [dB]")
axs[0].set_xlim(xlimit)
axs[0].set_ylim([40, 100])
axs[0].legend(loc=3)


# angles_gest = np.unwrap(np.angle(g_est)) # unwrap the angle to make it continuous; not applied for this figure
# angles_paz = np.unwrap(np.angle(freq_response_forward))
angles_gest = np.angle(g_est)
angles_paz = np.angle(freq_response_forward)

axs[1].grid(True, which="both", ls="-", alpha=0.2)

axs[1].semilogx(g_freqs, np.rad2deg(angles_gest), 'k', label="Estimated with Y/U")
axs[1].semilogx(freqs, np.rad2deg(angles_paz), 'r', label="Modeled with Poles and Zeros")

axs[1].set_xlabel("Frequency [Hz]")
axs[1].set_ylabel('Phase [deg]')
axs[1].set_ylim([-180, 180])
axs[1].set_yticks([-180, -90, 0, 90, 180])
axs[1].set_xlim(xlimit)
axs[1].legend(loc=3)

plt.tight_layout()
# plt.savefig(figdir+"/AEsensor_bode.png", dpi=300)


In [ ]:
ax

# Read gain factor obtained from ball drop test

In [ ]:
df_gain = pd.read_csv(fi_balldropgain, index_col=0)
df_gain.loc[df_gain['AEsensor'] == "OL01"]["gainfactor"].values # this includes 40dB of pre-amplification
df_gain

In [ ]:
df_gain[df_gain["AEsensor"] == "OL03"]["gainfactor"].values[0]

# Read response of preamp filter

In [ ]:
A = sio.loadmat(fi_preamp);
print(A.keys())

tvec = np.squeeze(A["tvec"]) # PAZ of the model for G20/G40
u_input_40 = np.squeeze(A["u_input_40"]) # PAZ of the model for G20/G40
y_output_20 = np.squeeze(A["y_output_20"]) # PAZ of the model for G20/G40
y_output_60 = np.squeeze(A["y_output_60"]) # PAZ of the model for G20/G40

PAZ_param = dict()

M20 = A["PAZ20"] # PAZ of the model for G20/G40
PAZ_param["p_20"] = np.squeeze(M20['p'][0][0])
PAZ_param["z_20"] = np.squeeze(M20['z'][0][0])
PAZ_param["k_20"] = np.squeeze(M20['k'][0][0])

M60 = A["PAZ60"] # PAZ of the model for G60/G40
PAZ_param["p_60"] = np.squeeze(M60['p'][0][0])
PAZ_param["z_60"] = np.squeeze(M60['z'][0][0])
PAZ_param["k_60"] = np.squeeze(M60['k'][0][0])


In [ ]:
# pickle event waveform directory
eventdatadir ='../data/04_2_AE_obs_isocoord_forFig2'

# greens function data directory 

greendatadir =  rootdir+'./05_numericalsimulation/case_gougepatch_fb03-087_cluster_cp6200_cs3600_v7_mergedcatalog/out/green/S00'

# output datadir
outputdir = f"../data/06_2_assemble_gf_model{balldrop_model}"

# channel table
channel_finame = '../../Others/AEchanneltable/AEsensorlocation_onFB03_table.csv'


In [ ]:
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

In [ ]:
# read gindex table
df_gindextable = pd.read_csv("../data/datacsv/gindex_table.csv", index_col=0)
df_gindextable

In [ ]:
# read observation casename  table
df_iso_all = pd.read_csv("../data/datacsv/AEevents_isocoord_forFig2.csv", index_col=0)
# reassemble event id 
event_cases = np.unique(df_iso_all["datacase"])
df_iso_all.head()

In [ ]:
event_cases

In [ ]:
# Read Channel Index
df_array = pd.read_csv(channel_finame)

channel_loc={}

for i in range(len(df_array)):
    stnm = df_array.iloc[i].Instrument_Label
    xtemp = df_array.iloc[i].North.astype('float')
    ytemp = df_array.iloc[i].East.astype('float')
    ztemp = df_array.iloc[i].Down.astype('float')
    channel_loc[stnm] = [xtemp, ytemp, ztemp]
    
AEsensors = list(channel_loc.keys())

## Assemble observation and synthetic waveform

In [ ]:
# Read observation
pre_filt = (1e4, 2e4, 1e6, 2e6) # prefilter for the remove_resp
water_level = 60 # waterlevel [dB] for the remove_resp

for event_case in tqdm(event_cases):
# event_case = event_cases[1]

    foname = os.path.join(outputdir, "{}_AEwaveform_forFig2.pickle".format(event_case)) 

    # if os.path.exists(foname):
    #     print("{}_bdwaveform.pickle already exists. skip this case.".format(event_case))
    #     continue

    df_iso_st = df_iso_all[df_iso_all.datacase==event_case]
    xlimit = [-0.5, 3]
    ylimit = [-1.0, 1.0]

    # if not df_iso_st:
    #     print("{}_bdwaveform.pickle already exists. skip this case.".format(stnm))
    #     continue

    df_iso_st

    st_eventsta = Stream()

    #---Start assembling each AE event---#
    for dataind in df_iso_st.index:
        #---Read observation data---#
        stnm, runID, event_id = dataind.split('__')

        tr_obs_trim = read(eventdatadir+"/obs_isocoord_{}_forFig2.pickle".format(dataind))[0]
        tr_obs_trim.stats.dataindex = dataind
        # rename channel
        tr_obs_trim.stats.channel = 'OZ' # 'OZ: Observatoin in Z direction with sidecoord_seg

        #--------------------------------------------------#
        #---Remove the instrumental response---#
        #--------------------------------------------------#
        # Stage 3: deconvolve preamp filter (G60/G40)
        # Stage 2: deconvolve AE sensor + preamp +40dB
        # Stage 1: rescale with the ball-drop calibration: NOTE: the aperture effect should be removed during the waveform fitting
        
        # Stage 3:
        if preamp_dB == 60:
            tr_stage3 = remove_resp_digitalfilt(tr_obs_trim, PAZ_param["p_60"], PAZ_param["z_60"], PAZ_param["k_60"],
                                            pre_filt=pre_filt, water_level=water_level, zero_mean=True, taper=True, taper_fraction=0.05, detrend=True, debug=False)
        elif preamp_dB == 20:
            tr_stage3 = remove_resp_digitalfilt(tr_obs_trim, PAZ_param["p_20"], PAZ_param["z_20"], PAZ_param["k_20"],
                                            pre_filt=pre_filt, water_level=water_level, zero_mean=True, taper=True, taper_fraction=0.05, detrend=True, debug=False)
            
        else:
            # no application for stage 3
            tr_stage3 = tr_obs_trim
            
        # Stage 2:
        tr_stage2 = remove_resp_digitalfilt(tr_stage3, poles_AE, zeros_AE, scale_fac_AE, # update: u_normfact is already included in scale_fac_AE
                                            pre_filt=pre_filt, water_level=water_level, zero_mean=True, taper=True, taper_fraction=0.05, detrend=True, debug=False)
        
        # Stage 1: 
        tr_stage1 = tr_stage2.copy()
        sensorgain = df_gain[df_gain["AEsensor"] == stnm]["gainfactor"].values[0]
        print(sensorgain)
        tr_stage1.data = tr_stage2.data/sensorgain
        tr_stage1.stats.location="stage1"
        st_eventsta.append(tr_stage1)

        
        # append to event stream both with and without instrumental response removal
        tr_obs_trim_raw = tr_obs_trim.copy()
        tr_obs_trim_raw.stats.location="raw"
        st_eventsta.append(tr_obs_trim_raw)

        tr_obs_trim_stage3 = tr_stage3.copy()
        tr_obs_trim_stage3.stats.location="stage3"
        st_eventsta.append(tr_obs_trim_stage3)

        tr_obs_trim_stage2 = tr_stage2.copy()
        tr_obs_trim_stage2.stats.location="stage2"
        st_eventsta.append(tr_obs_trim_stage2)
        #--------------------------------------------------#

        # Skip the synthetic green's functions
#         #---read green's tensor---#
#         # read synthetic green's function
#         gid = df_gindextable[df_gindextable['index'] == dataind]['gid'].values[0]
#         finame_syn = '{}/{}__{:08d}__S00__z__m*__.sac'.format(greendatadir, simulationtitle, gid) # Green's tensor in [nm/s]
#         print(finame_syn, gid)
#         trs_syn = read(finame_syn, format="SAC")

#         # restore stats
#         for tr_syn in trs_syn:
#             tr_syn.stats.network = 'OL'
#             tr_syn.stats.station = stnm
#             tr_syn.stats.location = 'GF'
#         #     tr_syn.stats.channel = '' 
#             tr_syn.stats.dataindex = dataind

#             # rescale source scaling and convert from nm to m
#             tr_syn.data = tr_syn.data/1e9
#             # manipulate start and endtime
#             # 1. zero pad pretriger and windowlen
#             # 2. change starttime

#             T0 = 0.0 # Origin time used in OpenSWPC

#             st_tmp = tr_syn.stats.starttime
#             pt = timedelta(milliseconds = tr_obs_trim.stats.pretrigger)
#             wt = timedelta(milliseconds = tr_obs_trim.stats.windowlen)
#             ot = timedelta(milliseconds = T0)

#             starttime_tmp = st_tmp+ot-pt
#             endtime_tmp = st_tmp+ot+wt

#             tr_syn_trim = tr_syn.copy() # avoid error in multiple trim
#             tr_syn_trim.trim(starttime_tmp, endtime_tmp, pad=True, fill_value=0.0)
#             tr_syn_trim.stats.starttime =  tr_obs_trim.stats.starttime

#             st_eventsta.append(tr_syn_trim)

    # Save Stream containing a set of observation and synthetic waveform
    if not os.path.exists(outputdir):
        os.makedirs(outputdir) 
    st_eventsta.write(foname, format="pickle")

In [ ]:
tr_stage2.plot()

In [ ]:
# print(st_eventsta.__str__(extended=True))

In [ ]:
event_case

In [ ]:
# plot comparison with and without instrumental response removal
event_case = "fb03-087__0072"
stnm = "OL07"
finame = os.path.join(outputdir, "{}_AEwaveform_forFig2.pickle".format(event_case)) 
st_event = read(finame) # this contains observation and green's function within a thresholded distance
tr_wrem = st_event.select(station=stnm, location="stage1", channel="OZ")[0]
tr_norem = st_event.select(station=stnm, location="raw", channel="OZ")[0]

In [ ]:
print(st_event.__str__(extended=True))

In [ ]:
print(st_event.__str__(extended=True))

In [ ]:
# stnm_list = np.unique([tr.stats.station for tr in st_event])
# manually rearrange with distance from source
stnm_list = ["OL24", "OL08", "OL23", "OL07", "OL22", "OL06"]


In [ ]:
pretrigger = tr_wrem.stats

In [ ]:
Nsensor = len(stnm_list)
fig, axs = plt.subplots(Nsensor, 1, figsize=(12, 9), sharex=True)

freqmin = 6e4
freqmax = 6e5
xlimit = [0, 0.1]
ylimits =[0.2, 0.2, 0.4, 0.4, 0.2, 0.1] 
scale_fac = 5000 # emperical scaling factor; which is estimated from ball drop test

for i, stnm in enumerate(stnm_list):

    tr_wrem = st_event.select(station=stnm, location="stage1", channel="OZ")[0]
    tr_norem = st_event.select(station=stnm, location="raw", channel="OZ")[0]

    # apply bandpass filter

    tr_wrem_filtered = tr_wrem.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax, corners=2, zerophase=True)
    tr_norem_filtered = tr_norem.copy().filter("bandpass", freqmin=freqmin, freqmax=freqmax, corners=2, zerophase=True)

    # 1. plot entire trace
    tvec = (tr_wrem_filtered.times())*1e3 - tr_wrem_filtered.stats.pretrigger
    axs[i].plot(tvec, tr_norem_filtered.data*1e3/scale_fac, "k-")
    axs[i].plot(tvec, tr_wrem_filtered.data*1e3, "r-")
    axs[i].set_xlim(xlimit)

    # set ylim
    fs = tr_wrem_filtered.stats.delta 
    tmax_ind = np.min(np.where(tvec>xlimit[1]))
    ylim_amp_filt = 1.2*np.maximum(np.max(np.abs(tr_norem_filtered.data[:tmax_ind]*1e3)), np.max(np.abs(tr_wrem_filtered.data[:tmax_ind]*1e3)))
#     axs[i].set_ylim([-ylim_amp_filt, ylim_amp_filt])
    axs[i].set_ylim([-ylimits[i], ylimits[i]])
    print([-ylimits[i], ylimits[i]])
    axs[i].text(0.01, 0.8, f"{stnm}", transform=axs[i].transAxes)

axs[0].set_title(r"Event {}  Bandpass filtered: {:.0f}-{:.0f}kHz".format(event_case, freqmin/1e3, freqmax/1e3))
fig.text(0.0, 0.5, 'Velocity [mm/s]', va='center', rotation='vertical')

axs[-1].set_xlabel("Time [ms]")

axs[0].legend(["raw unscaled", "instrumental response removed"], bbox_to_anchor=(1.01, 1.0), loc="upper left")

plt.tight_layout()
plt.savefig(figdir+"/comparison_response_removal_event{}_forFig2.png".format(event_case), dpi=300)

        

In [ ]:
tr_obs_trim.stats.starttime